# Voltage and Circuits 1

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [1]*

>
>   Consider the circuit fragment in the figure below. 
>   (Let $I_1 = 7$ [A], $I_3 = 1$ [A], $I_4 = 6$ [A]). 
>
>   {{ graphic }}
>
>   __<p align='right'>(10 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (A)

>
>   __What is the absolute value of the outward-going conventional current $I_2$?__
>
>    &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{A}\right]$
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (B)

>
>   __In the previous case (part A), did we make the right guess about the direction of the conventional current $I_2$?__ (We assumed the current was positive.)
>
>   (select one:)
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (C)

>
>   Suppose $I_3$ is $22$ [A]; __what is the absolute value of the outward-going conventional current $I_2$?__
>
>    &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{A}\right]$
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (D)

>
>   __In the previous case (part C), did we make the right guess about the direction of the conventional current $I_2$?__ (We assumed the current was positive.)
>
>   (select one:)
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [2]

>
>   In the circuit below, the narrow resistor is made of the same material as the thick connecting wires. 
>
>   {{ graphic }}
>
>   In the steady state, which graphs correctly show the magnitude of the electric field at locations around the circuit, and the drift speed of the elctrons at locations around the circuit? 
>
>   __Select the two correct graphs in the following:__
>
>   {{ graphic | graphic }}
>
>   - [ ] &nbsp; (a)
>
>   - [ ] &nbsp; (b)
>
>   - [ ] &nbsp; (c)
>
>   - [ ] &nbsp; (d)
>
>   - [ ] &nbsp; (e)
>
>   - [ ] &nbsp; (f)
>
>   - [ ] &nbsp; (g)
>
>   - [ ] &nbsp; (h)
>
>   - [ ] &nbsp; (i)
>
>   - [ ] &nbsp; (j)
>
>   - [ ] &nbsp; (k)
>
>   - [ ] &nbsp; (l)
>
>
>   __<p align='right'>(0.1 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [3]

>
>   In the circuit shown below, a mechanical battery keeps a steady-state current, running in a wire which has rather low electron mobility. 
>
>   {{ graphic }}
>   
>   Which of the following statements about the circuit are true?
>
>   (select all that apply:)
>
>   - [ ] &nbsp; At location 3, the electric field points to the left. 
>
>   - [ ] &nbsp; At every location inside the wire, the direction of the electric field is parallel to the wire. 
>
>   - [ ] &nbsp; The non-zero electric field inside the wire is created by excess charges on the surface of the wire and in and on the mechanical battery. 
>
>   - [ ] &nbsp; The non-zero electric field inside the wire is created by the moving electrons in the wire. 
>
>   - [ ] &nbsp; Mobile electrons inside the wire push each other through the wire. 
>
>   - [ ] &nbsp; The electric field inside the wire varies in magnitude, depending on location. 
>
>   - [ ] &nbsp; The magnitude of the electric field inside the wire is the same at all locations inside the wire. 
>
>   - [ ] &nbsp; At location 3 inside the wire, the electric field points to the right.
>
>   - [ ] &nbsp; The electric field is zero at all locations inside the metal wire. 
>
>
>   __<p align='right'>(0.1 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [4]*

>
>   Inside a chemical battery, it is not actually individual electrons that are 
>   transported from the "$+$" end to the "$-$" end. 
>
>   At the "$+$" end of the battery, an "acceptor" molecule picks up an electron 
>   entering the battery; and at the "$-$" end, a different "donor" molecule 
>   gives up an electron (which leaves the battery). Ions, rather than electrons, 
>   move between the two ends to transport the charge inside the battery. 
>
>   When the supplies of acceptor and donor molecules are used up in a chemical 
>   battery, the battery is dead, because it can no longer accept or release 
>   electrons. The electron current (in electrons per second), times the number 
>   of seconds of battery life, is equal to the number of donor (or acceptor) 
>   molecules in the battery.
>
>   ---
>   
>   A flashlight battery contains approximately half a mole of donor molecules. 
>   The current through a thick-filament bulb, powered by two flashlight 
>   batteries in series, is about $0.262$ [A]. 
>
>   __About how many hours will the batteries keep this bulb lit?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{h}\right]$
>
>   __<p align='right'>(10 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [5]*

>
>   A Nichrome wire $29$ [cm] long, and $0.23$ [mm] in diameter is connected to
>   a $3.1$ [V] battery.
>
>   __<p align='right'>(10 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (A)

>
>   __What is the electric field inside the wire?__ (Enter the magnitude.)
>
>    &emsp;&emsp;&emsp;&emsp; $Q = \fbox{\hspace{8em}?}\ \left[\textrm{V}/\textrm{m}\right]$
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (B)

>
>   __Why don't you have to know how the wire is bent?__ 
>
>   - [ ] &nbsp; $E$ must be parallel to the wire at every location
>               - even if the wire twists and turns.
>
>   - [ ] &nbsp; Since conventional current flows in the direction of $\vec{E}$,
>               $E$ is the same in every part of the wire with uniform properties.
>                
>   - [ ] &nbsp; $E$ is not dependent on how the wire is bent, only the diameter
>               of the wire is needed.
>
>   - [ ] &nbsp; $E$ is dependent on length $L$ of the wire, which only applies 
>               to the segment of the wire that is not bent. 
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<hr width="82%" style="border-bottom:2px dashed; border-top:none"/>

### (B)

>
>   __How would your answer change, if the wire diameter were $0.42$ [mm]?__
>   (Note that the electric field in the wire is quite small compared to the 
>    electric field near a charged tape.)
>
>   - [ ] &nbsp; The answer will be larger.
>   - [ ] &nbsp; The answer will be smaller.
>   - [ ] &nbsp; The answer will stay the same. 
>

<hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 

---

## [6]

>
>   Consider the following figure.
>
>   {{ graphic }}
>
>   __What would be the potential difference $V_C - V_B$ across the thin resistor in the figure__,
>   if the battery $\mathcal{EMF}$ is $8.7$ [V]? 
>
>   Assume that the electric field in the thick wires is very small, (so that the
>   potential differences along the thick wires are negligible).
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>   
>   __Do you have enough information to determine the current $\boldsymbol{J}$ in the circuit?__
>
>   (select one:)
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   __<p align='right'>(0.1 points)</p>__
>

<hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

---